In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Define the CNN architecture without large kernel sizes
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # First block: Reduce dimensions gradually
        self.conv1 = nn.Conv2d(3, 10, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(10, 10, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Second block: Increase filters and reduce dimensions
        self.conv3 = nn.Conv2d(10, 35, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(35, 35, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Third block: Further reduction with stride and pooling
        self.conv5 = nn.Conv2d(35, 100, kernel_size=3, stride=2, padding=0)
        self.conv6 = nn.Conv2d(100, 100, kernel_size=3, stride=2, padding=0)
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=3)
        
        # Fully connected layers
        self.fc1 = nn.Linear(4 * 4 * 100, 512)  # Flattened size: 1600
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 100)  # 100 classes for CIFAR-100
        
        self.relu = nn.ReLU()
    
    def forward(self, x):
        # Input shape: [batch_size, 3, 250, 250]
        x = self.relu(self.conv1(x))  # [batch_size, 10, 250, 250]
        x = self.relu(self.conv2(x))  # [batch_size, 10, 250, 250]
        x = self.pool1(x)             # [batch_size, 10, 125, 125]
        x = self.relu(self.conv3(x))  # [batch_size, 35, 125, 125]
        x = self.relu(self.conv4(x))  # [batch_size, 35, 125, 125]
        x = self.pool2(x)             # [batch_size, 35, 62, 62]
        x = self.relu(self.conv5(x))  # [batch_size, 100, 30, 30]
        x = self.relu(self.conv6(x))  # [batch_size, 100, 14, 14]
        x = self.pool3(x)             # [batch_size, 100, 4, 4]
        x = x.view(x.size(0), -1)     # [batch_size, 1600]
        x = self.relu(self.fc1(x))    # [batch_size, 512]
        x = self.relu(self.fc2(x))    # [batch_size, 256]
        x = self.fc3(x)               # [batch_size, 100]
        return x

# Hyperparameters
batch_size = 64
epochs = 10
learning_rate = 0.001

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data preparation for CIFAR-100
transform = transforms.Compose([
    transforms.Resize((250, 250)),  # Resize to match original input size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

# Initialize the model, loss function, and optimizer
model = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_loss = running_loss / len(trainloader)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on test set: {accuracy:.2f}%')

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/10], Loss: 4.0110
Epoch [2/10], Loss: 3.3296
Epoch [3/10], Loss: 3.0101
Epoch [4/10], Loss: 2.7892
Epoch [5/10], Loss: 2.6210
Epoch [6/10], Loss: 2.4836
Epoch [7/10], Loss: 2.3673
Epoch [8/10], Loss: 2.2685
Epoch [9/10], Loss: 2.1632
Epoch [10/10], Loss: 2.0781
Accuracy on test set: 32.47%


In [13]:
import torch
import torchvision.models as models
from efficientnet_pytorch import EfficientNet
from torchvision import transforms
from PIL import Image
import requests
from io import BytesIO
import json

# Define device (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load pre-trained ResNet50 and EfficientNet-B0
resnet = models.resnet50(pretrained=True).to(device)
resnet.eval()  # Set to evaluation mode for inference

efficientnet = EfficientNet.from_pretrained('efficientnet-b0').to(device)
efficientnet.eval()  # Set to evaluation mode for inference

# Define image preprocessing (same for both models)
transform = transforms.Compose([
    transforms.Resize(256),            # Resize to 256x256
    transforms.CenterCrop(224),        # Crop to 224x224
    transforms.ToTensor(),             # Convert to tensor
    transforms.Normalize(              # Normalize with ImageNet mean and std
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

# Load ImageNet class labels from a hosted JSON file
LABELS_URL = 'https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json'
response = requests.get(LABELS_URL)
labels = json.loads(response.content)  # List of 1000 ImageNet class names

# Function to load an image from a URL or local path
def load_image(image_path):
    if image_path.startswith('http'):
        response = requests.get(image_path)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_path).convert('RGB')
    return image

# Function to predict the class of an image using ResNet50
def predict_resnet(image):
    image_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = resnet(image_tensor)
        _, predicted = torch.max(outputs, 1)
    return labels[predicted.item()]

# Function to predict the class of an image using EfficientNet-B0
def predict_efficientnet(image):
    image_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = efficientnet(image_tensor)
        _, predicted = torch.max(outputs, 1)
    return labels[predicted.item()]

# Example usage
image_path = 'https://upload.wikimedia.org/wikipedia/commons/4/47/PNG_transparency_demonstration_1.png'
image = load_image(image_path)

resnet_pred = predict_resnet(image)
efficientnet_pred = predict_efficientnet(image)

print(f"ResNet50 Prediction: {resnet_pred}")
print(f"EfficientNet-B0 Prediction: {efficientnet_pred}")


Loaded pretrained weights for efficientnet-b0
ResNet50 Prediction: salt shaker
EfficientNet-B0 Prediction: barrette


In [12]:
import torch
from PIL import Image
import requests
from io import BytesIO

# Load pre-trained YOLOv5s model (small version) from Ultralytics
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Function to load an image from a URL or local path
def load_image(image_path):
    if image_path.startswith('http'):  # If URL
        response = requests.get(image_path)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:  # If local path
        image = Image.open(image_path).convert('RGB')
    return image

# Example usage
image_path = 'https://imgd.aeplcdn.com/1200x900/n/cw/ec/159231/swift-right-front-three-quarter.jpeg?isig=0&q=80'  # Replace with your image URL or local path
image = load_image(image_path)

# Perform inference with YOLOv5
results = yolo_model(image)

# Print detected objects (class, confidence, and coordinates)
results.print()

# Show image with bounding boxes in a window
results.show()

# Save image with bounding boxes (saved to 'runs/detect/exp' by default)
results.save()

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16484 sha256=d989775b117ad09229cb5046f2489300815fff6f595c1466d2caffabafbf15ba
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\29\16\24\752e89d88d333af39a288421e64d613b5f652918e39ef1f8e3
Successfully built efficientnet_pytorch



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
